## Analysis Ready Sentinel-1 Backscatter Imagery with Intake

In [1]:
import datetime

import intake
import pandas as pd
import xarray as xr
from distributed import Client

In [2]:
client = Client(processes=True)
client

Client Scheduler: tcp://127.0.0.1:62819 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.00 GiB


In [3]:
!cat sentinel-1-aws-catalog.yaml

sources:
  sentinel_1_aws:
    description: |
      Analysis Ready Sentinel-1 Backscatter Imagery. Documentation -->
      https://sentinel-s1-rtc-indigo-docs.s3-us-west-2.amazonaws.com/data_format.html#data-structure
    driver: rasterio
    parameters:
      utm_zone:
        description: UTM Zone component of tile ID
        type: int
        default: 14
        min: 10 
        max: 19
      latitude_band:
        description: Latitude band (CONUS spans from R to U)
        type: str 
        default: T
        allowed:
          - R
          - S
          - T
          - U
      grid_square_id:
        description: The 100x100km Grid Square ID
        type: str 
        default: PN
        allowed:
          - BE
          - BF 
          - BG
          - KK
          - PN
      year:
        description: Year of the acquisition
        type: int
        default: 2020
        min: 2016
        max: 2021
      month:
          description:  Month of the acquisition
          type:

In [4]:
cat = intake.open_catalog("sentinel-1-aws-catalog.yaml")

In [5]:
cat.sentinel_1_aws()

sentinel_1_aws:
  args:
    chunks: {}
    storage_options:
      anon: true
    urlpath: s3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2020/S1A_20200801_14TPN_ASC/Gamma0_VV.tif
  description: 'Analysis Ready Sentinel-1 Backscatter Imagery. Documentation -->

    https://sentinel-s1-rtc-indigo-docs.s3-us-west-2.amazonaws.com/data_format.html#data-structure

    '
  driver: intake_xarray.raster.RasterIOSource
  metadata:
    catalog_dir: /Users/abanihi/devel/andersy005/intake-tutorial/notebooks/


In [6]:
cat.sentinel_1_aws(day=22, year=2021, month=4)

sentinel_1_aws:
  args:
    chunks: {}
    storage_options:
      anon: true
    urlpath: s3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2021/S1A_20210422_14TPN_ASC/Gamma0_VV.tif
  description: 'Analysis Ready Sentinel-1 Backscatter Imagery. Documentation -->

    https://sentinel-s1-rtc-indigo-docs.s3-us-west-2.amazonaws.com/data_format.html#data-structure

    '
  driver: intake_xarray.raster.RasterIOSource
  metadata:
    catalog_dir: /Users/abanihi/devel/andersy005/intake-tutorial/notebooks/


In [7]:
# Invalid parameters
cat.sentinel_1_aws(day=1, year=2010, month=6)

ValueError: year=2010 is less than 2016

In [8]:
cat.sentinel_1_aws(orbit_direction='test')

ValueError: orbit_direction=test is not one of the allowed values: ASC,DSC

In [9]:
%%time
# Load data
ds = cat.sentinel_1_aws(day=22, year=2021, month=4).to_dask()
ds

CPU times: user 365 ms, sys: 99.5 ms, total: 465 ms
Wall time: 1.27 s


<xarray.DataArray (band: 1, y: 5490, x: 5490)>
dask.array<open_rasterio-9403685fb237c1ed3eda6ae5d3f7d660<this-array>, shape=(1, 5490, 5490), dtype=float32, chunksize=(1, 5490, 5490), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 4.8e+06 4.8e+06 4.8e+06 ... 4.69e+06 4.69e+06 4.69e+06
  * x        (x) float64 6e+05 6e+05 6.000e+05 ... 7.098e+05 7.098e+05 7.098e+05
Attributes: (12/20)
    transform:              (20.0, 0.0, 600000.0, 0.0, -20.0, 4800000.0)
    crs:                    +init=epsg:32614
    res:                    (20.0, 20.0)
    is_tiled:               1
    nodatavals:             (0.0,)
    scales:                 (1.0,)
    ...                     ...
    OVR_RESAMPLING_ALG:     AVERAGE
    SCENES:                 S1A_IW_GRDH_1SDV_20210422T003023_20210422T003048_...
    SCENE_1_METADATA:       {"title": "S1A_IW_GRDH_1SDV_20210422T003023_20210...
    SCENE_1_PRODUCT_INFO:   {"id": "S1A_IW_GRDH_1SDV_20210422T003023_20210422...
    TILE_ID:                14TPN
    VALID_PIXEL_PERCENT:    34.363

In [10]:
date_range = pd.date_range(start='2021-02-01', end='2021-04-01')


# Function for cleaning the data: rename band -> time and create datetime object
def preprocess(ds):
    ds["band"] = [datetime.datetime.fromisoformat(ds.attrs["DATE"])]
    ds = ds.rename({'band': 'time'})
    return ds


def retrieve_dataset(value):
    try:
        ds = cat.sentinel_1_aws(year=value.year, month=value.month, day=value.day).to_dask()
        return preprocess(ds)
    except Exception:
        return None


datasets = client.map(retrieve_dataset, date_range)
datasets = client.gather(datasets)
datasets = [dataset for dataset in datasets if dataset is not None]
ds = xr.concat(datasets, dim='time', compat='override', coords='minimal')
ds

<xarray.DataArray (time: 8, y: 5490, x: 5490)>
dask.array<concatenate, shape=(8, 5490, 5490), dtype=float32, chunksize=(1, 5490, 5490), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2021-02-04 2021-02-09 ... 2021-03-29
  * y        (y) float64 4.8e+06 4.8e+06 4.8e+06 ... 4.69e+06 4.69e+06 4.69e+06
  * x        (x) float64 6e+05 6e+05 6.000e+05 ... 7.098e+05 7.098e+05 7.098e+05
Attributes: (12/20)
    transform:              (20.0, 0.0, 600000.0, 0.0, -20.0, 4800000.0)
    crs:                    +init=epsg:32614
    res:                    (20.0, 20.0)
    is_tiled:               1
    nodatavals:             (0.0,)
    scales:                 (1.0,)
    ...                     ...
    OVR_RESAMPLING_ALG:     AVERAGE
    SCENES:                 S1A_IW_GRDH_1SDV_20210204T002201_20210204T002226_...
    SCENE_1_METADATA:       {"title": "S1A_IW_GRDH_1SDV_20210204T002201_20210...
    SCENE_1_PRODUCT_INFO:   {"id": "S1A_IW_GRDH_1SDV_20210204T002201_20210204...
    TILE_ID:                14TPN
    VALID_PIXEL_PERCENT:    37.273

In [12]:
import hvplot.xarray

ds.isel(time=0).hvplot.image(rasterize=True, aspect='equal', cmap='gray', clim=(0, 0.4))

:DynamicMap   []
   :Image   [x,y]   (value)